### Importing the libraries 

In [8]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from dotenv import load_dotenv
import os

In [9]:
# Loading environment variables from .env file
load_dotenv()

# Getting the API key from environment variables
api_key = os.getenv('IATI_API_KEY')

# API endpoint
url = 'https://api.iatistandard.org/datastore/activity/iati'

# parameters 
params = {
    'q': '*:*',  # Query all records
    'rows': 5 ,   # Limit the number of results
    'fq': 'iati_identifier:"CH-4-1980001679"',    # Filter for specific identifier
}

# headers
headers = {
    'Accept': 'application/json',
    'Ocp-Apim-Subscription-Key': api_key 
}

# Making the GET request with parameters and headers
response = requests.get(url, params=params, headers=headers)

# response content for debugging
print(response.content)

if response.status_code == 200:
    try:
        # Parsing the XML response
        root = ET.fromstring(response.content)
        
        # Extracting data from XML
        activities = []
        for activity in root.findall('.//iati-activity'):
            activity_data = {
                'identifier': activity.find('iati-identifier').text if activity.find('iati-identifier') is not None else None,
                'title': activity.find('.//title/narrative').text if activity.find('.//title/narrative') is not None else None,
                'description': activity.find('.//description/narrative').text if activity.find('.//description/narrative') is not None else None,
                'status': activity.find('.//activity-status').attrib.get('code') if activity.find('.//activity-status') is not None else None,
                'start_date': activity.find('.//activity-date[@type="1"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="1"]') is not None else None,
                'end_date': activity.find('.//activity-date[@type="4"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="4"]') is not None else None,
                'reporting_org': activity.find('.//reporting-org/narrative').text if activity.find('.//reporting-org/narrative') is not None else None,
                'recipient_country': activity.find('.//recipient-country/narrative').text if activity.find('.//recipient-country/narrative') is not None else None,
                'sector': activity.find('.//sector/narrative').text if activity.find('.//sector/narrative') is not None else None,
                'budget': activity.find('.//budget/value').text if activity.find('.//budget/value') is not None else None,
                'transaction_type': activity.find('.//transaction/transaction-type').attrib.get('code') if activity.find('.//transaction/transaction-type') is not None else None,
                'transaction_date': activity.find('.//transaction/transaction-date').attrib.get('iso-date') if activity.find('.//transaction/transaction-date') is not None else None,
                'transaction_value': activity.find('.//transaction/value').text if activity.find('.//transaction/value') is not None else None,
                'transaction_description': activity.find('.//transaction/description/narrative').text if activity.find('.//transaction/description/narrative') is not None else None,
                'provider_org': activity.find('.//transaction/provider-org/narrative').text if activity.find('.//transaction/provider-org/narrative') is not None else None,
                'receiver_org': activity.find('.//transaction/receiver-org/narrative').text if activity.find('.//transaction/receiver-org/narrative') is not None else None,
                'finance_type': activity.find('.//transaction/finance-type').attrib.get('code') if activity.find('.//transaction/finance-type') is not None else None
            }
            activities.append(activity_data)
        
        # Converting to DF
        df = pd.DataFrame(activities)
        display(df)
    except ET.ParseError as e:
        print("Error parsing XML:", e)
else:
    print(f"Error: {response.status_code}")


b'<?xml version="1.0" encoding="UTF-8"?><iati-activities generated-datetime="2024-07-22T20:28:41.822Z" version="2.03"><iati-activity xml:lang="en" default-currency="CHF" last-updated-datetime="2020-03-04T00:00:00Z">\n      <iati-identifier>CH-4-1980001679</iati-identifier>\n      <reporting-org ref="CH-4" type="10">\n         <narrative>Swiss Agency for Development and Cooperation (SDC)</narrative>\n      </reporting-org>\n      <title>\n         <narrative>EPFZ Stages de perfectionnement</narrative>\n      </title>\n      <description type="1">\n         <narrative>No description for this activity available at this point. Description indisponible pour cette activit\xc3\xa9.</narrative>\n      </description>\n      <participating-org ref="CH-4" role="1" type="10">\n         <narrative>Swiss Agency for Development and Cooperation (SDC)</narrative>\n      </participating-org>\n      <participating-org ref="XM-DAC-51000" role="4" type="80">\n         <narrative>Universit\xc3\xa9, institut

,identifier,title,description,status,start_date,end_date,reporting_org,recipient_country,sector,budget,transaction_type,transaction_date,transaction_value,transaction_description,provider_org,receiver_org,finance_type
0,CH-4-1980001679,EPFZ Stages de perfectionnement,No description for this activity available at ...,2,2019-12-01,2023-11-30,Swiss Agency for Development and Cooperation (...,None,None,None,2,2019-12-15,0,Outgoing Commitment,None,None,None


pagination, since even if i delete row limitation on the code above I get max 10 rows.

In [16]:
# Load environment variables from .env file
load_dotenv()

# Get the API key from environment variables
api_key = os.getenv('IATI_API_KEY')

# API endpoint
url = 'https://api.iatistandard.org/datastore/activity/iati'

# Function to fetch data with pagination
def fetch_data(api_key, url, rows_per_page=100, max_pages=10):
    all_activities = []
    start = 0
    for page in range(max_pages):
        # Parameters with pagination
        params = {
            'q': '*:*',  # Query all records
            'rows': rows_per_page,  # Number of results per page
            'start': start,  # Starting point for each page
            'fq': [
                #'iati_identifier:"CH-4-1980001679"',  # Exact match filter for specific identifier
                'reporting_org_ref:"CH-4"'  # Exact match filter for organization identifier
            ]
        }

        # Headers
        headers = {
            'Accept': 'application/json',
            'Ocp-Apim-Subscription-Key': api_key 
        }

        # Making the GET request with parameters and headers
        response = requests.get(url, params=params, headers=headers)

        if response.status_code == 200:
            try:
                # Parsing the XML response
                root = ET.fromstring(response.content)

                # Extracting data from XML
                activities = []
                for activity in root.findall('.//iati-activity'):
                    activity_data = {
                        'identifier': activity.find('iati-identifier').text if activity.find('iati-identifier') is not None else None,
                        'title': activity.find('.//title/narrative').text if activity.find('.//title/narrative') is not None else None,
                        'description': activity.find('.//description/narrative').text if activity.find('.//description/narrative') is not None else None,
                        'status': activity.find('.//activity-status').attrib.get('code') if activity.find('.//activity-status') is not None else None,
                        'start_date': activity.find('.//activity-date[@type="1"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="1"]') is not None else None,
                        'end_date': activity.find('.//activity-date[@type="4"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="4"]') is not None else None,
                        'reporting_org': activity.find('.//reporting-org/narrative').text if activity.find('.//reporting-org/narrative') is not None else None,
                        'recipient_country': activity.find('.//recipient-country/narrative').text if activity.find('.//recipient-country/narrative') is not None else None,
                        'sector': activity.find('.//sector/narrative').text if activity.find('.//sector/narrative') is not None else None,
                        'budget': activity.find('.//budget/value').text if activity.find('.//budget/value') is not None else None,
                        'transaction_type': activity.find('.//transaction/transaction-type').attrib.get('code') if activity.find('.//transaction/transaction-type') is not None else None,
                        'transaction_date': activity.find('.//transaction/transaction-date').attrib.get('iso-date') if activity.find('.//transaction/transaction-date') is not None else None,
                        'transaction_value': activity.find('.//transaction/value').text if activity.find('.//transaction/value') is not None else None,
                        'transaction_description': activity.find('.//transaction/description/narrative').text if activity.find('.//transaction/description/narrative') is not None else None,
                        'provider_org': activity.find('.//transaction/provider-org/narrative').text if activity.find('.//transaction/provider-org/narrative') is not None else None,
                        'receiver_org': activity.find('.//transaction/receiver-org/narrative').text if activity.find('.//transaction/receiver-org/narrative') is not None else None,
                        'finance_type': activity.find('.//transaction/finance-type').attrib.get('code') if activity.find('.//transaction/finance-type') is not None else None
                    }
                    activities.append(activity_data)

                all_activities.extend(activities)

                # Increment the start for the next page
                start += rows_per_page

                # Break if no more results
                if len(activities) < rows_per_page:
                    break

            except ET.ParseError as e:
                print("Error parsing XML:", e)
                break
        else:
            print(f"Error: {response.status_code}")
            break

    return all_activities

# Fetching data
activities = fetch_data(api_key, url, rows_per_page=100, max_pages=10)

# Converting to DataFrame
df_ch4 = pd.DataFrame(activities)


Error: 429


In [17]:
df_ch4 

,identifier,title,description,status,start_date,end_date,reporting_org,recipient_country,sector,budget,transaction_type,transaction_date,transaction_value,transaction_description,provider_org,receiver_org,finance_type
0,CH-4-2020002175,Crédit global pour petites actions,Les crédits globaux pour petites actions perme...,2,2019-06-01,None,Swiss Agency for Development and Cooperation (...,Tanzanie (République-Unie),None,None,2,2020-12-31,0,Outgoing Commitment,None,None,None
1,CH-4-2020002185,Crédit global pour petites actions,Les crédits globaux pour petites actions perme...,2,2017-07-01,None,Swiss Agency for Development and Cooperation (...,Macédoine (Ex-République yougoslave de),None,None,2,2020-12-31,0,Outgoing Commitment,None,None,None
2,CH-4-2020002197,Crédit global pour petites actions,Les crédits globaux pour petites actions perme...,2,2016-03-15,None,Swiss Agency for Development and Cooperation (...,Kosovo,None,None,2,2020-12-31,0,Outgoing Commitment,None,None,None
3,CH-4-2020002205,Crédit global pour petites actions,Les crédits globaux pour petites actions perme...,2,2019-03-01,None,Swiss Agency for Development and Cooperation (...,Bosnie-Herzégovine,None,None,2,2020-12-31,50000,Outgoing Commitment,None,None,None
4,CH-4-2020002208,Crédit global pour petites actions,Les crédits globaux pour petites actions perme...,2,2019-03-01,None,Swiss Agency for Development and Cooperation (...,Bosnie-Herzégovine,None,None,2,2020-12-31,270370,Outgoing Commitment,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CH-4-2008004889,"World Food Programme (WFP), food assistance",In line with the Good Humanitarian Donorship (...,2,2020-01-01,None,Swiss Agency for Development and Cooperation (...,Myanmar,None,None,2,2020-12-31,2500000,Outgoing Commitment,None,None,None
96,CH-4-2008004890,"World Food Programme (WFP), food assistance",In line with the Good Humanitarian Donorship (...,2,2020-01-01,None,Swiss Agency for Development and Cooperation (...,Afghanistan,None,None,2,2020-12-31,1000000,Outgoing Commitment,None,None,None
97,CH-4-2008004891,"World Food Programme (WFP), food assistance",In line with the Good Humanitarian Donorship (...,2,2020-01-01,None,Swiss Agency for Development and Cooperation (...,Afghanistan,None,None,2,2020-12-31,1000000,Outgoing Commitment,None,None,None
98,CH-4-2008004892,"World Food Programme (WFP), food assistance",In line with the Good Humanitarian Donorship (...,2,2020-01-01,None,Swiss Agency for Development and Cooperation (...,Colombie,None,None,2,2020-12-31,1000000,Outgoing Commitment,None,None,None


In [14]:
load_dotenv()


api_key = os.getenv('IATI_API_KEY')

# API endpoint
url = 'https://api.iatistandard.org/datastore/activity/iati'

# Function to fetch data with pagination
def fetch_data(api_key, url, rows_per_page=100, max_pages=10):
    all_activities = []
    start = 0
    for page in range(max_pages):
        # Parameters with pagination
        params = {
            'q': '*:*',  
            'rows': rows_per_page,  
            'start': start, 
            # 'fq': [
            #     'iati_identifier:"CH-4-1980001679"',  
            #     'reporting_org_ref:"CH-4"'  
            # ]
        }

        # Headers
        headers = {
            'Accept': 'application/json',
            'Ocp-Apim-Subscription-Key': api_key 
        }

        # Making the GET request with parameters and headers
        response = requests.get(url, params=params, headers=headers)

        if response.status_code == 200:
            try:
                # Parsing the XML response
                root = ET.fromstring(response.content)

                # Extracting data from XML
                activities = []
                for activity in root.findall('.//iati-activity'):
                    activity_data = {
                        'identifier': activity.find('iati-identifier').text if activity.find('iati-identifier') is not None else None,
                        'title': activity.find('.//title/narrative').text if activity.find('.//title/narrative') is not None else None,
                        'description': activity.find('.//description/narrative').text if activity.find('.//description/narrative') is not None else None,
                        'status': activity.find('.//activity-status').attrib.get('code') if activity.find('.//activity-status') is not None else None,
                        'start_date': activity.find('.//activity-date[@type="1"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="1"]') is not None else None,
                        'end_date': activity.find('.//activity-date[@type="4"]').attrib.get('iso-date') if activity.find('.//activity-date[@type="4"]') is not None else None,
                        'reporting_org': activity.find('.//reporting-org/narrative').text if activity.find('.//reporting-org/narrative') is not None else None,
                        'recipient_country': activity.find('.//recipient-country/narrative').text if activity.find('.//recipient-country/narrative') is not None else None,
                        'sector': activity.find('.//sector/narrative').text if activity.find('.//sector/narrative') is not None else None,
                        'budget': activity.find('.//budget/value').text if activity.find('.//budget/value') is not None else None,
                        'transaction_type': activity.find('.//transaction/transaction-type').attrib.get('code') if activity.find('.//transaction/transaction-type') is not None else None,
                        'transaction_date': activity.find('.//transaction/transaction-date').attrib.get('iso-date') if activity.find('.//transaction/transaction-date') is not None else None,
                        'transaction_value': activity.find('.//transaction/value').text if activity.find('.//transaction/value') is not None else None,
                        'transaction_description': activity.find('.//transaction/description/narrative').text if activity.find('.//transaction/description/narrative') is not None else None,
                        'provider_org': activity.find('.//transaction/provider-org/narrative').text if activity.find('.//transaction/provider-org/narrative') is not None else None,
                        'receiver_org': activity.find('.//transaction/receiver-org/narrative').text if activity.find('.//transaction/receiver-org/narrative') is not None else None,
                        'finance_type': activity.find('.//transaction/finance-type').attrib.get('code') if activity.find('.//transaction/finance-type') is not None else None
                    }
                    activities.append(activity_data)

                all_activities.extend(activities)

                # Increment the start for the next page
                start += rows_per_page

                # Break if no more results
                if len(activities) < rows_per_page:
                    break

            except ET.ParseError as e:
                print("Error parsing XML:", e)
                break
        else:
            print(f"Error: {response.status_code}")
            break

    return all_activities


activities = fetch_data(api_key, url, rows_per_page=100, max_pages=10)

# Converting to DataFrame
df = pd.DataFrame(activities)



Error: 429


In [15]:
#df

,identifier,title,description,status,start_date,end_date,reporting_org,recipient_country,sector,budget,transaction_type,transaction_date,transaction_value,transaction_description,provider_org,receiver_org,finance_type
0,XI-IATI-WBTF-P104385,Institutional Strengthening of the Legal Claim...,The overall development objective of the prop...,3,2006-04-27,2011-04-04,World Bank Trust Funds,COLOMBIA,+![cdata[Anti-corruption organisations and ins...,478491.39,11,2020-06-30,478491.39,Incoming Commitment (Total Grant Allocated Amo...,Institutional Development Fund (IDF),COLOMBIA,110
1,XI-IATI-WBTF-P104395,Legal Education Capacity Building Project,This IDF will suport the Nigeria Legal Educait...,3,2007-06-20,2011-11-22,World Bank Trust Funds,NIGERIA,+![cdata[Legal and judicial development]]+,440062.74,11,2020-06-30,440062.74,Incoming Commitment (Total Grant Allocated Amo...,Institutional Development Fund (IDF),NIGERIA,110
2,XI-IATI-WBTF-P104397,HN Enhancing Transparency and Accountability i...,The Transparency law that was approved on Nove...,3,2007-04-19,2011-09-08,World Bank Trust Funds,HONDURAS,+![cdata[Anti-corruption organisations and ins...,332717.02,11,2020-06-30,332717.02,Incoming Commitment (Total Grant Allocated Amo...,Institutional Development Fund (IDF),HONDURAS,110
3,XI-IATI-WBTF-P104398,Strengthening Monitoring and Evaluation in Brazil,Soound monitoring and evaluation of programs a...,3,2008-03-26,2011-08-26,World Bank Trust Funds,BRAZIL,+![cdata[Public sector policy and administrati...,155000.00,11,2020-06-30,155000.00,Incoming Commitment (Total Grant Allocated Amo...,Institutional Development Fund (IDF),BRAZIL,110
4,XI-IATI-WBTF-P104400,HT - Improving Health Surveillance,Capacity Building of Ministry of Health on He...,3,2009-04-29,2011-10-11,World Bank Trust Funds,HAITI,+![cdata[Health policy and administrative mana...,93212.00,11,2020-06-30,93212.00,Incoming Commitment (Total Grant Allocated Amo...,Institutional Development Fund (IDF),HAITI,110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,XI-IATI-WBTF-P111185,SL Strengthening Parliamentary Oversight Commi...,"Description of the IDF - Financed Components,...",3,2008-07-14,2012-01-17,World Bank Trust Funds,SRI LANKA,+![cdata[Public finance management]]+,137383.81,11,2020-06-30,137383.81,Incoming Commitment (Total Grant Allocated Amo...,Institutional Development Fund (IDF),SRI LANKA,110
96,XI-IATI-WBTF-P111283,Ministry of Trade & Economic Development Capac...,The objective of this project is to strengthe...,3,2008-10-28,2012-03-02,World Bank Trust Funds,ARMENIA,+![cdata[Industrial development]]+,387360.06,11,2020-06-30,387360.06,Incoming Commitment (Total Grant Allocated Amo...,Institutional Development Fund (IDF),ARMENIA,110
97,XI-IATI-WBTF-P111310,Improving Management over Special Purpose Tran...,The overall development objective of the propo...,3,2008-07-10,2012-02-09,World Bank Trust Funds,CHINA,+![cdata[Financial policy and administrative m...,171781.31,11,2020-06-30,171781.31,Incoming Commitment (Total Grant Allocated Amo...,Institutional Development Fund (IDF),CHINA,110
98,XI-IATI-WBTF-P111320,WAEMU Development of Accounting Regulation Ins...,The objective is to support regional coordina...,3,2010-06-07,2014-02-26,World Bank Trust Funds,None,+![cdata[Public sector policy and administrati...,855513.24,11,2020-06-30,855513.24,Incoming Commitment (Total Grant Allocated Amo...,Institutional Development Fund (IDF),"Africa, regional",110
